# running EC2 commands via boto3

Boto3 SSM documentation:
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ssm.html#SSM.Client.send_command

AWS EC2 instances:
https://console.aws.amazon.com/ec2/v2/home?region=us-east-1#Instances:

Couple of gotchas while POCing:
  * required install of SSM on EC2: https://docs.aws.amazon.com/systems-manager/latest/userguide/agent-install-al2.html
  * required IAM permissions for SSM: https://docs.aws.amazon.com/systems-manager/latest/userguide/setup-instance-profile.html
  * seemed to require a reboot

In [2]:
import time
import boto3

In [3]:
# init boto3 SSM client
client = boto3.client('ssm', region_name='us-east-1')

In [5]:
# send hello world
results = client.send_command(
    InstanceIds=["i-07a0c73b0ceb6ce87"],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": ["echo 'hello world'"]},
)
display(results)

{'Command': {'CommandId': '61d70d45-9932-4bff-95df-a097bfa17094',
  'DocumentName': 'AWS-RunShellScript',
  'DocumentVersion': '$DEFAULT',
  'Comment': '',
  'ExpiresAfter': datetime.datetime(2022, 1, 16, 15, 45, 40, 358000, tzinfo=tzlocal()),
  'Parameters': {'commands': ["echo 'hello world'"]},
  'InstanceIds': ['i-07a0c73b0ceb6ce87'],
  'Targets': [],
  'RequestedDateTime': datetime.datetime(2022, 1, 16, 13, 45, 40, 358000, tzinfo=tzlocal()),
  'Status': 'Pending',
  'StatusDetails': 'Pending',
  'OutputS3Region': 'us-east-1',
  'OutputS3BucketName': '',
  'OutputS3KeyPrefix': '',
  'MaxConcurrency': '50',
  'MaxErrors': '0',
  'TargetCount': 1,
  'CompletedCount': 0,
  'ErrorCount': 0,
  'DeliveryTimedOutCount': 0,
  'ServiceRole': '',
  'NotificationConfig': {'NotificationArn': '',
   'NotificationEvents': [],
   'NotificationType': ''},
  'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
   'CloudWatchOutputEnabled': False},
  'TimeoutSeconds': 3600},
 'ResponseMetadata': 

In [ ]:
results

In [6]:
# get results of command (noting this won't be needed mostly)
results = client.get_command_invocation(
    InstanceId="i-07a0c73b0ceb6ce87",
    CommandId="4c43e368-523f-4c06-a35e-e4a4de27b857"
)
display(results)

{'CommandId': '4c43e368-523f-4c06-a35e-e4a4de27b857',
 'InstanceId': 'i-07a0c73b0ceb6ce87',
 'Comment': '',
 'DocumentName': 'AWS-RunShellScript',
 'DocumentVersion': '$DEFAULT',
 'PluginName': 'aws:runShellScript',
 'ResponseCode': 0,
 'ExecutionStartDateTime': '2022-01-15T08:06:21.422Z',
 'ExecutionElapsedTime': 'PT0.038S',
 'ExecutionEndDateTime': '2022-01-15T08:06:21.422Z',
 'Status': 'Success',
 'StatusDetails': 'Success',
 'StandardOutputContent': 'hello world\n',
 'StandardOutputUrl': '',
 'StandardErrorContent': '',
 'StandardErrorUrl': '',
 'CloudWatchOutputConfig': {'CloudWatchLogGroupName': '',
  'CloudWatchOutputEnabled': False},
 'ResponseMetadata': {'RequestId': '5579256e-0045-4ea5-ab89-1541e8360f6e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 16 Jan 2022 12:46:15 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '595',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5579256e-0045-4ea5-ab89-1541e8360f6e

## notes on using SSM for issuing Kafka CLI commands on EC2 

First pass at running Kafka CLI command failed due to following permission error:

```
An error occurred (AccessDeniedException) when calling the DescribeCluster operation: User: arn:aws:sts::007385147054:assumed-role/SSMInstanceProfile/i-07a0c73b0ceb6ce87 is not authorized to perform: kafka:DescribeCluster on resource: arn:aws:kafka:us-east-1:007385147054:cluster/poc-cluster-1/7a7a5d89-0d30-4a3c-94ab-b6c4f18c0fc0-21\nfailed to run commands: exit status 255
```

This was resolved by granting more IAM permissions to the profile -- `arn:aws:iam::007385147054:role/SSMInstanceProfile` -- that was granted to the ECS machine that allowed SSM commands. 

In [8]:
# run Kafka CLI command; polling for results
# prepare command string
cmd = 'aws kafka describe-cluster --region us-east-1 --cluster-arn "arn:aws:kafka:us-east-1:007385147054:cluster/poc-cluster-1/7a7a5d89-0d30-4a3c-94ab-b6c4f18c0fc0-21"'

# send command
cmd_send_results = client.send_command(
    InstanceIds=["i-07a0c73b0ceb6ce87"],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": [cmd]},
)

# wait 2 seconds (improve w/ polling?)
time.sleep(2)

# retrieve results of SSM command and print stdout
cmd_get_results = client.get_command_invocation(
    InstanceId="i-07a0c73b0ceb6ce87",
    CommandId=cmd_send_results['Command']['CommandId']
)
display(cmd_get_results['StandardOutputContent'])

'{\n    "ClusterInfo": {\n        "LoggingInfo": {\n            "BrokerLogs": {\n                "S3": {\n                    "Enabled": false\n                }, \n                "Firehose": {\n                    "Enabled": false\n                }, \n                "CloudWatchLogs": {\n                    "Enabled": true, \n                    "LogGroup": "/msk/poc/events"\n                }\n            }\n        }, \n        "EncryptionInfo": {\n            "EncryptionInTransit": {\n                "ClientBroker": "TLS_PLAINTEXT", \n                "InCluster": true\n            }, \n            "EncryptionAtRest": {\n                "DataVolumeKMSKeyId": "arn:aws:kms:us-east-1:007385147054:key/edf7751a-2f70-44c6-845e-e5e7ec3e0954"\n            }\n        }, \n        "BrokerNodeGroupInfo": {\n            "BrokerAZDistribution": "DEFAULT", \n            "ClientSubnets": [\n                "subnet-3fca7158", \n                "subnet-1d8c0a33"\n            ], \n            "Stor

In [7]:
# testing speed of sending SSM messages

def test_ssm_cmds():
    t0 = time.time()
    for x in range(0,10):
        t1 = time.time()
        results = client.send_command(
            InstanceIds=["i-07a0c73b0ceb6ce87"],
            DocumentName="AWS-RunShellScript",
            Parameters={"commands": ["echo 'hello world'"]},
            CloudWatchOutputConfig={
                "CloudWatchOutputEnabled":True,
                "CloudWatchLogGroupName":"/aws/ssm/runcommand/test1"
            }
        )
        print(f"{x} elapsed: {time.time()-t1}")
    print(f"total elapsed: {time.time()-t0}")

test_ssm_cmds()

0 elapsed: 0.5674312114715576
1 elapsed: 0.21296000480651855
2 elapsed: 0.2064208984375
3 elapsed: 0.1791551113128662
4 elapsed: 0.21007490158081055
5 elapsed: 0.22102999687194824
6 elapsed: 0.22236299514770508
7 elapsed: 0.14899611473083496
8 elapsed: 0.19582176208496094
9 elapsed: 0.2171766757965088
total elapsed: 2.402070999145508


## Notes for function above...

The use of `CloudWatchOutputConfig` allows for writing of stdout to Cloudwatch

In [4]:
# testing parsing of multiple arguments
cmd_send_results = client.send_command(
    InstanceIds=["i-07a0c73b0ceb6ce87"],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": [
        """python -c "import sys; import json; msg = json.loads(sys.argv[1]); print(json.dumps(msg));" '{"n1":"this is a JSON document","n2":true, "n3":{"n4":false, "n5":42}}'"""
    ]},
    CloudWatchOutputConfig={
        "CloudWatchOutputEnabled":True,
        "CloudWatchLogGroupName":"/aws/ssm/runcommand/test1"
    }
)